# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [1]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
dsms = DSMS(env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [ ]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.Specimen,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.Specimen,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties={"Geometry": "Cylindrical 150mm x 20mm x 40mm",
                       "Material": "Concrete",
                       "Project ID": "ConstructionProject2024"
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties={"Geometry": "Rectangular 200mm x 30mm x 20mm",
                       "Material": "Metal",
                       "Project ID": "MetalBlenders2024"
                       }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1,item2],
    annotations=["www.researchBACiri.org/foo"],
)

dsms.commit()

/app/dsms/knowledge/kitem.py:575: UserWarning: A flat dictionary was provided for custom properties.
                    Will be transformed into `KItemCustomPropertiesModel`.
  warnings.warn(



<b> <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>    </b> 


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [7]:
result = dsms.search(ktypes=[dsms.ktypes.Specimen], limit=2)
print(result)
print("Name of the first kitem:")
print(result.hits[0].kitem.name)

Results:
hits:
- kitem:
    name: Specimen-2
    id: 8cb51ac4-4d05-4bc1-b9f0-9d54333f7d50
    ktype_id: specimen
    slug: specimen-2-8cb51ac4
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestPiece
      label: TestPiece
      namespace: https://w3id.org/steel/ProcessOntology
    attachments: []
    linked_kitems:
    - id: bdeebac2-a14c-44fb-9277-637e9c3d4036
      name: Machine-2
      slug: machine-2-bdeebac2
      ktype_id: material
      summary: null
      avatar_exists: false
      annotations:
      - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
        label: TestingMachine
        namespace: https://w3id.org/steel/ProcessOntology
      linked_kitems:
      - id: 8cb51ac4-4d05-4bc1-b9f0-9d54333f7d50
      - id: 2b769463-8f9b-4479-a525-6819abd22124
      external_links: []
      contacts: []
      authors:
      - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
      linked_affiliations: []
      attachments: []
      user_groups: []
      cust

... and for all of type `Organization` and `DatasetCatalog`:

In [ ]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog], limit=1):
    print(result.kitem)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

kitem:
  name: Research Institute ABC
  id: 109e910b-e8dd-42ac-8b41-91f85e4b1be4
  ktype_id: organization
  slug: researchinstituteabc-109e910b
  annotations:
  - iri: www.researchBACiri.org/foo
    label: foo
    namespace: www.researchBACiri.org
  attachments: []
  linked_kitems:
  - id: 67f59cdb-38ae-4a22-996e-2bb33348e7c9
    name: Machine-1
    slug: machine-1-67f59cdb
    ktype_id: material
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: 79e87341-4714-46dc-a0ee-7a3f3ab9d160
    - id: 109e910b-e8dd-42ac-8b41-91f85e4b1be4
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []
    custom_properties:
      sections:
      - id: id1733827827

... or for all of type `Dataset` with `Specimen-1` in the name:

In [6]:
for result in dsms.search(query="Specimen-1", ktypes=[dsms.ktypes.Dataset], limit=1):
    print(result.hit)
    print("\n")

... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [7]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], limit=1
    ):
    print(result.hit)
    print("\n")

KItem(

	name = Research Institute ABC, 

	id = 2335f57a-efc2-47ce-83b3-866cdeb6d30d, 

	ktype_id = organization, 

	in_backend = True, 

	slug = researchinstituteabc-2335f57a, 

	annotations = [
		{
			iri: www.researchBACiri.org/foo,
			name: research ABC Institute,
			namespace: research,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: d1524272-5f34-454b-bb61-7b0ab3a825a7
			name: Machine-1
			slug: machine-1-d1524272
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: ba1a7871-ff75-465f-8625-a7a0202edca4
		}, {
			id: 2335f57a-efc2-47ce-83b3-866cdeb6d30d
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-23T17:15:45.431622
		

## 5.3. Fetching linked KItems from a KItem

In the beginning under **5.1** we created some kitems and linked each other. Now we want to fetch the linked kitems and display them to the user. For this we use the `linked_kitems` attribute.

In [8]:
item5.linked_kitems

[
		
			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
			name: Machine-1
			slug: machine-1-31a7d354
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
		}, {
			id: 379e07a7-acd3-45dd-8bbd-ad5edb549946
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-23T18:16:40.453930
			updated_at: 2024-08-23T18:16:40.453930
		, 
		
			id: 8155c264-c790-4151-bff5-471e6a64fc21
			name: Machine-2
			slug: machine-2-8155c264
			ktype_id: testing-machine
			summary: None
			avatar

However, this linked KItem is not a "real" KItem. Due to performance reasons, we only display a slim representation of the linked KItem. Imagine if the linked KItem also has a linked KItem... this might lead to an infinite recursion. We can fetch the "real" KItem by using the `fetch` method of the linked KItem:


In [9]:
item5.linked_kitems[0].fetch()

KItem(

	name = Machine-1, 

	id = 31a7d354-6dfc-4aa5-832c-a1e581757193, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-31a7d354, 

	annotations = [
		{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
			name: Specimen-1
			slug: specimen-1-af4e0334
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestPiece,
			name: TestPiece,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sample

We can also get the linked KItems grouped by annotation:

In [10]:
item5.linked_kitems.by_annotation

{'https://w3id.org/steel/ProcessOntology/TestingMachine': [
  			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
  			name: Machine-1
  			slug: machine-1-31a7d354
  			ktype_id: testing-machine
  			summary: None
  			avatar_exists: False
  			annotations: [{
  			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
  			name: TestingMachine,
  			namespace: https://w3id.org/steel/ProcessOntology,
  			description: None
  		}]
  			linked_kitems: [{
  			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
  		}, {
  			id: 379e07a7-acd3-45dd-8bbd-ad5edb549946
  		}]
  			external_links: []
  			contacts: []
  			authors: [{
  			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  		}]
  			linked_affiliations: []
  			attachments: []
  			user_groups: []
  			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
  			created_at: 2024-08-23T18:16:40.453930
  			updated_at: 2024-08-23T18:16:40.453930
  		,
  
  			id: 8155c264-c790-4151-bff5-4

Again, we are able to fetch the "real" KItem by using the `fetch` method of the linked KItem:

In [11]:
item5.linked_kitems.by_annotation["https://w3id.org/steel/ProcessOntology/TestingMachine"][0].fetch()

KItem(

	name = Machine-1, 

	id = 31a7d354-6dfc-4aa5-832c-a1e581757193, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-31a7d354, 

	annotations = [
		{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
			name: Specimen-1
			slug: specimen-1-af4e0334
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestPiece,
			name: TestPiece,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sample

Additionally, we can retrieve the linked KItems grouped by ktype:

In [12]:
item5.linked_kitems.by_ktype

{<KTypes.TestingMachine: 'testing-machine'>: [
  			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
  			name: Machine-1
  			slug: machine-1-31a7d354
  			ktype_id: testing-machine
  			summary: None
  			avatar_exists: False
  			annotations: [{
  			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
  			name: TestingMachine,
  			namespace: https://w3id.org/steel/ProcessOntology,
  			description: None
  		}]
  			linked_kitems: [{
  			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
  		}, {
  			id: 379e07a7-acd3-45dd-8bbd-ad5edb549946
  		}]
  			external_links: []
  			contacts: []
  			authors: [{
  			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  		}]
  			linked_affiliations: []
  			attachments: []
  			user_groups: []
  			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
  			created_at: 2024-08-23T18:16:40.453930
  			updated_at: 2024-08-23T18:16:40.453930
  		,
  
  			id: 8155c264-c790-4151-bff5-471e6a64fc21
 

... and retrieve them by type:

In [13]:
item5.linked_kitems.by_ktype[dsms.ktypes.TestingMachine][0].fetch()

KItem(

	name = Machine-1, 

	id = 31a7d354-6dfc-4aa5-832c-a1e581757193, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-31a7d354, 

	annotations = [
		{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: af4e0334-6325-4ae7-91fd-cc1b21864cf1
			name: Specimen-1
			slug: specimen-1-af4e0334
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestPiece,
			name: TestPiece,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: 31a7d354-6dfc-4aa5-832c-a1e581757193
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sample

Clean up the DSMS from the tutortial:

In [14]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()